### Import Libraries

In [1]:
import requests
import json
import pandas as pd

### Initialise Functions

In [2]:
#function to extract data from btcmarkets
def crypto_price_btcmarkets(crypto_ticker,btcmarkets_api):

    # Gather prices
    crypto_bid = btcmarkets_api["bestBid"]
    crypto_ask = btcmarkets_api["bestAsk"]
    crypto_last = btcmarkets_api["lastPrice"]
    
    # print(f"{crypto_ticker} Buy: ${float(crypto_bid):,}")
    # print(f"{crypto_ticker} Sell: ${float(crypto_ask):,}")
    # print(f"{crypto_ticker} Last Transaction: ${float(crypto_last):,}")
    
    #return as dictionary
    return{"btcmarkets bid":crypto_bid,"btcmarkets ask":crypto_ask,"btcmarkets sell":crypto_last}

#function to extract data from btcmarkets
def crypto_price_coinspot(crypto_ticker):
    
    # Gather prices
    crypto_bid = coinspot_data["prices"][crypto_ticker]["bid"]
    crypto_ask = coinspot_data["prices"][crypto_ticker]["ask"]
    crypto_last = coinspot_data["prices"][crypto_ticker]["last"]
    
    # print(f"{crypto_ticker} Buy: ${float(crypto_bid):,}")
    # print(f"{crypto_ticker} Sell: ${float(crypto_ask):,}")
    # print(f"{crypto_ticker} Last Transaction: ${float(crypto_last):,}")

        #return as dictionary
    return{"coinspot bid":crypto_bid,"coinspot ask":crypto_ask,"coinspot sell":crypto_last}

### Execute GET request with URL

def get_api(coin_url):

    # Execute get request
    requests.get(coin_url)

    ### Store response as variable

    # Execute GET request and store response
    coin_response_data = requests.get(coin_url)

    ### Retrieve API output using `content` attribute

    # Get content
    coin_response_content = coin_response_data.content

    # print(response_content)

    # Formatting as json
    coin_data = coin_response_data.json()
    
    #Print data as formatted json with indents
    # print(json.dumps(coinspot_data, indent=4))
    
    return coin_data

### Initialise API calls

In [3]:
# Create variable to hold request url
coinspot_url = "https://www.coinspot.com.au/pubapi/latest"
btcmarkets_url_btc ="https://api.btcmarkets.net/v3/markets/BTC-AUD/ticker"
btcmarkets_url_eth ="https://api.btcmarkets.net/v3/markets/ETH-AUD/ticker"

### GET API Data

In [4]:
# Return the json file using get_api function
coinspot_data = get_api(coinspot_url)
btcmarkets_data_btc = get_api(btcmarkets_url_btc)
btcmarkets_data_eth = get_api(btcmarkets_url_eth)
### Format JSON with indents


### Convert to Dictionaries

In [5]:
# Run Coinspot Functions
btc_coinspot = crypto_price_coinspot(crypto_ticker = "btc")
eth_coinspot = crypto_price_coinspot(crypto_ticker = "eth")

#Run btcmarkets functions
btc_btcmarkets = crypto_price_btcmarkets(crypto_ticker = "btc",btcmarkets_api = btcmarkets_data_btc)
eth_btcmarkets = crypto_price_btcmarkets(crypto_ticker = "eth",btcmarkets_api = btcmarkets_data_eth)

#Join Dictionaries
btcmarkets_prices = {"btc": btc_btcmarkets,"eth" :eth_btcmarkets}
coinspot_prices = {"btc": btc_coinspot,"eth" :eth_coinspot}

### Create DataFrame

In [6]:
# Convert Dictionaries to Dataframe
coinspot_df = pd.DataFrame.from_dict(coinspot_prices,orient = "index")
btcmarkets_df = pd.DataFrame.from_dict(btcmarkets_prices,orient = "index")

# Join dataframes
all_prices = pd.concat([coinspot_df,btcmarkets_df], axis="columns", join="inner")

#Show Data
all_prices.head()

,coinspot bid,coinspot ask,coinspot sell,btcmarkets bid,btcmarkets ask,btcmarkets sell
btc,28800,29186.69999975,29521.37,28714.71,28762.37,28727.31
eth,1560,1561.30190491,1563.65562541,1552.07,1555.62,1557.89


### Convert to float and perform basic calculations

In [7]:
#Show Data
all_prices = all_prices.astype(float).copy()

all_prices["bid_diff"] = all_prices["coinspot bid"] - all_prices["btcmarkets bid"]
all_prices["ask_diff"] = all_prices["coinspot ask"] - all_prices["btcmarkets ask"]
all_prices["sell_diff"] = all_prices["coinspot sell"] - all_prices["btcmarkets sell"]

all_prices

,coinspot bid,coinspot ask,coinspot sell,btcmarkets bid,btcmarkets ask,btcmarkets sell,bid_diff,ask_diff,sell_diff
btc,28800.0,29186.700000,29521.370000,28714.71,28762.37,28727.31,85.29,424.330000,794.060000
eth,1560.0,1561.301905,1563.655625,1552.07,1555.62,1557.89,7.93,5.681905,5.765625
